In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from scipy import stats
import strava_prepare


In [3]:
df = strava_prepare.get_strava_data()

In [ ]:
df

In [ ]:
df = strava_prepare.typ_clean(df)

In [4]:
df.shape

(1937, 8)

In [ ]:
df.to_csv('strava.csv', index=False)
